# Задание 1

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [3]:
import nltk
import re
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aomak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [7]:
def find_jaccard(definition, context):
    intersection = (set(definition.split()) & set(context.split()))
    union = (set(definition.split()) | set(context.split()))
    jaccard =  len(intersection) / len(union)
    return jaccard

def lesk_algorithm_jaccard(meaning, context):
    the_word = re.split('%', meaning)[0]
    definitions = {}
    for synset in wn.synsets(the_word):
        definitions[synset.definition()] = find_jaccard(synset.definition(), context)
    max_one = [0, '']
    for definition in definitions:
        if definitions[definition] > max_one[0]:
            max_one[0] = definitions[definition]
            max_one[1] = definition
    return max_one

In [36]:
correct_definitions = 0
sum_of_words = 0
for sent in corpus_wsd:
    if len(sent) > 1:
        context = ' '.join([word[2] for word in sent])
        
        for word in sent:
            if '%' in word[0]:
                context = context.replace(word[2], '_')
                lesk_definition = lesk_algorithm_jaccard(word[0], context)
                if lesk_definition[1] == wn.lemma_from_key(word[0]).synset().definition():
                    correct_definitions+=1
                sum_of_words+=1

print(correct_definitions/sum_of_words)

0.224077061268043


# Задание 2
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты. 